In [1]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Fashion-MNIST的均值和标准差与MNIST不同，这里保持不变
])

# 加载Fashion-MNIST数据集
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 定义LeNet-1模型
class LeNet1(nn.Module):
    def __init__(self):
        super(LeNet1, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 实例化两个LeNet-1模型
model_selector = LeNet1().to(device)  # 用于选择最有价值5%测试用例的模型
model_predictor = LeNet1().to(device)  # 用于对5%测试用例进行预测的模型

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer_selector = optim.Adam(model_selector.parameters(), lr=0.001)  # 选择模型的优化器
optimizer_predictor = optim.Adam(model_predictor.parameters(), lr=0.001)  # 预测模型的优化器

# 训练模型
def train_model(model, dataloader, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# 训练选择模型
train_model(model_selector, train_loader, optimizer_selector)  # 训练选择模型

# 训练预测模型
train_model(model_predictor, train_loader, optimizer_predictor)  # 训练预测模型在训练集上
train_model(model_predictor, test_loader, optimizer_predictor)  # 训练预测模型在测试集上

# 评估模型并计算置信度
model_selector.eval()
uncertainties = []
labels = []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        output = model_selector(x)  # 使用选择模型进行评估
        probabilities = torch.softmax(output, dim=1)
        max_prob, _ = torch.max(probabilities, dim=1)
        uncertainties.extend(max_prob.cpu().numpy())
        labels.extend(y.cpu().numpy())

# 选择最不自信的5%测试用例
threshold_index = int(len(uncertainties) * 0.05)
threshold = np.partition(uncertainties, -threshold_index)[threshold_index]
selected_indices = np.where(uncertainties < threshold)[0]

# 使用预测模型对挑选出的测试用例进行预测
correct_count = 0
total_count = 0
with torch.no_grad():
    for idx in selected_indices:
        img, label = test_dataset[idx]  # 获取图片和标签
        img = img.unsqueeze(0).to(device)  # 增加批次维度并移动到设备
        output = model_predictor(img)  # 使用预测模型进行预测
        _, predicted = torch.max(output, 1)
        if predicted.item() == label:
            correct_count += 1
        total_count += 1

# 计算正确分类率
accuracy = correct_count / total_count
print(f"正确分类率: {accuracy:.4f}")

Epoch [1/5], Loss: 0.3764
Epoch [2/5], Loss: 0.4397
Epoch [3/5], Loss: 0.2750
Epoch [4/5], Loss: 0.4661
Epoch [5/5], Loss: 0.3143
Epoch [1/5], Loss: 0.3505
Epoch [2/5], Loss: 0.2805
Epoch [3/5], Loss: 0.3821
Epoch [4/5], Loss: 0.3389
Epoch [5/5], Loss: 0.2632
Epoch [1/5], Loss: 0.2519
Epoch [2/5], Loss: 0.2011
Epoch [3/5], Loss: 0.1757
Epoch [4/5], Loss: 0.1580
Epoch [5/5], Loss: 0.1437
正确分类率: 0.8130
